In [1]:
import logging
import numpy as np  
from gensim.models import Doc2Vec
import os
import codecs

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)

WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


In [69]:
import re 
import string
import unicodedata
import sys

STOPWORDS = [u"wwDATEww", u"wwTIMEww", u"wwEMAILww", u"wwIPww", u"wwURLww", u"wwNUMBERww"]

tbl = dict.fromkeys(i for i in xrange(sys.maxunicode)
                      if unicodedata.category(unichr(i)).startswith('P') and i != 45 and i!= 95)

def vi_trans_unicode(su):
    return su.translate(tbl)

vpunctuation = string.punctuation.replace('-','').replace('_','')
print vpunctuation

def vi_strip_text(s):
    s = re.sub(r"&amp;", "", s)
    s = re.sub(u"\u2026", "", s, re.UNICODE) # ...
    s = re.sub(u"\u2014", "", s, re.UNICODE) # emdash
    s = re.sub(u"\u201d", "", s, re.UNICODE) #
    s = re.sub(u"\u201c", "", s, re.UNICODE) #
    s = re.sub(r"<([^>]+)>", "", s)
    s = re.sub(r"(\s|\\n|\\r|\\t)+", " ", s)
    s = re.sub("([%s]+)" % vpunctuation, " ", s)
    s = ' '.join([w if not w[0].isdigit() else u"_NUM" for w in s.strip().split()])
    return s

def vi_strip_text2(s):
    s = re.sub(r"&amp;", "", s)
    s = vi_trans_unicode(s)
    s = re.sub(r"<([^>]+)>", "", s)
    s = re.sub(r"(\s|\\n|\\r|\\t)+", " ", s)
    s = re.sub(r"__+", "_", s)
    s = re.sub(r"--+", "-", s)
    s = ' '.join([w if not w[0].isdigit() else u"_NUM" for w in s.strip().split()])
    return s

def vi_clean(line):
    words = line.replace('.',' ').strip().split()
    words = [w for w in words if not (w in STOPWORDS) and not (u"_NUM" in w) and not (w.startswith(u'-'))]
    return ' '.join(words)

!"#$%&'()*+,./:;<=>?@[\]^`{|}~


In [3]:
import requests
from bs4 import BeautifulSoup

url_post = "http://vlsp.vietlp.org:8080/demo/?page=seg_pos_chunk"
header_post = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/4",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8"
}

def tokenize_soup(soup):
    try:
        sentences = soup.findAll('div', {'class': "sentence"})
        toks = [ s.find('tr').findAll('td')[1:] for s in sentences]
        toks = [t.contents[0] for tok in toks for t in tok]
        return ' '.join(toks)
    except Exception, e:
        print e
        
def get_tokens(INPUT):
    try:
        payload = {r"input":INPUT}
        rsp = requests.post(url_post, data=payload, headers=header_post)
        soup = BeautifulSoup(rsp.content)
        return tokenize_soup(soup)
    except Exception, e:
        print e

In [4]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [5]:
model = Doc2Vec.load("/home/laampt/orm/10K/5K/model/vinews_tok_doc585K_dm_5ns_200features_2minwords_8context_e5_casen_cpu_3epo_alpha_r131")

INFO:gensim.utils:loading Doc2Vec object from /home/laampt/orm/10K/5K/model/vinews_tok_doc585K_dm_5ns_200features_2minwords_8context_e5_casen_cpu_3epo_alpha_r131
INFO:gensim.utils:loading docvecs recursively from /home/laampt/orm/10K/5K/model/vinews_tok_doc585K_dm_5ns_200features_2minwords_8context_e5_casen_cpu_3epo_alpha_r131.docvecs.* with mmap=None
INFO:gensim.utils:loading doctag_syn0 from /home/laampt/orm/10K/5K/model/vinews_tok_doc585K_dm_5ns_200features_2minwords_8context_e5_casen_cpu_3epo_alpha_r131.docvecs.doctag_syn0.npy with mmap=None
INFO:gensim.utils:loading syn1neg from /home/laampt/orm/10K/5K/model/vinews_tok_doc585K_dm_5ns_200features_2minwords_8context_e5_casen_cpu_3epo_alpha_r131.syn1neg.npy with mmap=None
INFO:gensim.utils:loading syn0 from /home/laampt/orm/10K/5K/model/vinews_tok_doc585K_dm_5ns_200features_2minwords_8context_e5_casen_cpu_3epo_alpha_r131.syn0.npy with mmap=None
INFO:gensim.utils:setting ignored attribute syn0norm to None
INFO:gensim.utils:setting ign

In [6]:
print model

Doc2Vec(dm/s,d200,n5,w8,mc2,s1e-05,t4)


In [7]:
model.init_sims(replace=True)

INFO:gensim.models.word2vec:precomputing L2-norms of word weight vectors


In [8]:
model.docvecs['r131_PD_0'].shape

(200,)

In [9]:
for w,v in model.most_similar([u"ô-liu"]):
    print w, v

ô_liu 0.864648461342
oliu 0.846231639385
hạnh_nhân 0.832529187202
ôliu 0.829264581203
Cà_chua 0.819293379784
bí_đỏ 0.811026573181
cần_tây 0.810770452023
Cà_rốt 0.809538602829
dâu_tây 0.804501652718
olive 0.801929056644


In [10]:
for w,v in model.most_similar([u"phản_động"]):
    print w, v

chống_phá 0.860351741314
xuyên_tạc 0.824813902378
Việt_Tân 0.792726516724
tay_sai 0.780650615692
mưu_đồ 0.763554632664
vu_khống 0.751157402992
bắt_bớ 0.742017626762
ngôn_luận 0.740430057049
thâm_độc 0.73834246397
vu_cáo 0.737820327282


In [11]:
print model.docvecs['r131_WK_0'].shape
print model.docvecs['r131_PD_0'].shape
print model.docvecs['r131_ZN_0'].shape
print model.docvecs['r131_TN_0'].shape
print model.docvecs['r131_VN_0'].shape

(200,)
(200,)
(200,)
(200,)
(200,)


In [12]:
vn_127 = u"""Sau khi được mở_rộng cầu Bình Triệu sẽ góp_phần giảm áp_lực lưu_thông qua_cầu Sài_Gòn tăng nhanh khả_năng thông_thương giữa TP HCM và các tỉnh miền Đông_Nam_Bộ Cũng nằm trong dự_án BOT cầu_đường Bình Triệu _NUM cầu Bình Triệu mới đã được đưa vào sử_dụng cuối tháng _NUM và cho xe lưu_thông theo chiều từ Bình_Dương về TP HCM Ông Trần Quang Phượng Giám_đốc Sở Giao_thông vận_tải TP HCM đánh_giá cầu Bình Triệu sau gần _NUM năm khai_thác đã dần xuống_cấp và có nguy_cơ sập bất cứ lúc nào nên việc trùng_tu là hết sức quan_trọng Theo đó CII đơn_vị chủ đầu_tư đã thuê nhà_thầu nước_ngoài là Công_ty Freyssinet Pháp nâng_cấp cầu cũ từ tải_trọng _NUM tấn lên thành _NUM tấn với _NUM làn xe _NUM làn dành cho ôtô và một làn dành cho xe hai bánh Kiên_Cường Ảnh Kiên_Cường Dự_án BOT cầu_đường Bình Triệu _NUM được khởi_động vào ngày _NUM gồm xây_dựng cầu Bình Triệu mới và sửa_chữa mở_rộng cầu cũ nhưng vì một_số lý_do khách_quan nên mới chỉ xây được cầu mới Dự_kiến dự_án sửa_chữa sẽ hoàn_thành trong _NUM tháng và thời_gian khai_thác là _NUM năm với tổng giá_trị khoảng gần _NUM t đồng Đơn_vị thi_công cam_kết trong thời_gian làm mới các phương_tiện vẫn lưu_thông bình_thường ở _NUM cầu theo hướng cũ từ TP HCM đi Bình_Dương Chúng_tôi sẽ thay toàn_bộ khe co_giãn cùng các gối cầu trải lại lớp nhựa làm hệ_thống chiếu_sáng Cầu cũng được mở_rộng từ _NUM m lên_NUM m Đồng_thời cầu cũng sẽ có khả_năng chịu động_đất cấp _NUM sau khi hoàn_thành đại_diện Công_ty Freyssinet nói về phương_án k thuật sửa_chữa Hôm_nay chúng_tôi sẽ triển_khai việc sữa chữa nâng_cấp và mở_rộng cầu cũ ông Lê Vũ Hoàng Tổng giám_đốc Công_ty cổ_phần đầu_tư hạ_tầng k thuật TP HCM CII cho_biết"""
vn_127 = vi_clean(vn_127)
vn_127_vec = model.infer_vector(vn_127.split())
for tag,v in model.docvecs.most_similar([vn_127_vec]):
    print tag,v

INFO:gensim.models.doc2vec:precomputing L2-norms of doc weight vectors


r131_TN_122645 0.597650170326
r131_VN_84801 0.593913137913
r131_VN_133173 0.59174156189
r131_TN_159596 0.585798680782
r131_VN_43747 0.575877308846
r131_VN_149899 0.574281573296
r131_TN_202910 0.57169675827
r131_TN_53710 0.571644186974
r131_TN_180849 0.571370840073
r131_TN_165892 0.566232085228


In [13]:
tn_127 = u"""Ý_định trồng ba_kích chợt lóe lên trong đầu người đàn_ông người C Tu này Ông nhiệt_tình cho giống hướng_dẫn cụ_thể cách trồng thu_hoạch với niềm_tin một ngày không xa sẽ có nhiều người miền_núi thoát nghèo vươn lên khá_giả đầy_đủ như mình Còn bây_giờ sau _NUM năm phát_triển và khá phổ_biến giá một ký ba_kích tươi đã lên đến _NUM đồng vậy mà còn không đủ để bán Ở xã Lăng nơi Bhríu Pố ở cũng đã có hơn _NUM gia_đình học_tập trồng ba_kích Ngoài dùng làm các vị_thuốc rượu ba_kích với nhiều tính_năng tuyệt_vời đã được nhiều người ưa_thích Hằng_ngày ông vẫn chân trần lội_suối vượt đồi để chăm_bẵm đồi ba_kích của mình Đằng_đẵng thời_gian dài đến lúc ba_kích lên tươi_tốt Bhríu Pố mới gọi bà_con đến xem Nhiều người vẫn nói quả đồi canh_tác của Bhríu Pố là đồi _NUM trong _NUM Lúc này mọi người mới tin cây của trời vẫn trồng được Mùa_thu hoạch ba_kích thường vào mùa_xuân tiết_trời mát Sau hơn _NUM năm gặp lại Bhríu Pố hào_hứng khoe đợt thu_hoạch vừa rồi ông trúng lớn Có hôm trời mưa nước lớn Bhríu Pố phải ngủ lại rừng Ông quyết_định vượt núi vào rừng đào ba_kích ròng_rã mấy tháng trời để về trồng TS Ngô Trại bảo đây là giống cây quý lắm có củ ở dưới người dân gọi là cây ruột gà Bây_giờ bất k ai đã từng đặt_chân đến vùng_đất Tây_Giang cũng đều cố_gắng có được một hũ rượu ba_kích mang về Với giá_thành hiện_tại chỉ làm một phép_tính sơ_sơ cũng đủ thấy đồi ba_kích của Bhríu Pố đang có giá_trị gần nửa tỉ đồng một giấc_mơ của nhiều người Làm_giàu nhờ trồng cây hiếm Thoát nghèo nhờ cây của trời _NUM Tôi Viết Chuyện Bhríu Pố giàu lên nhờ ba_kích loại cây quý_hiếm trong ngành dược_liệu VN trở_nên nổi_tiếng không chỉ ở Tây_Giang Quảng_Nam mà còn khắp các huyện miền_núi miền_Trung Chủ_tịch UBND huyện Tây_Giang Bhríu Liếc bộc_bạch Đất làm rẫy của người dân ngày_càng khó_khăn trong khi đó hiệu_quả kinh_tế của ba_kích thì thấy rõ nên huyện đã chủ_trương xác_định ba_kích sẽ là cây xóa đói giảm nghèo Năm _NUM lần đầu_tiên ba_kích tím được tìm thấy ở xã Lăng Tây_Giang Quảng_Nam trong chuyến đi thực_tế để tìm_kiếm các cây thuốc_nam của tiến_sĩ TS Ngô Trại Theo Giám_đốc Trung_tâm Y_tế huyện bác_sĩ Nguyễn Huy Thông huyện đã trồng được hơn _NUM gốc ở thôn PrNing Lúc trước khi ba_kích mới được tìm thấy một_vài người dân đi rừng đào được bán với giá _NUM đồng kg đến khi Bhríu Pố trồng được ba_kích giá đã nâng lên _NUM đồng kg Từ _NUM gốc ban_đầu hiện_nay số gốc ba_kích của Bhríu Pố đã lên đến hơn _NUM gốc Bhríu Pố nhớ như_in Mình cùng TS Ngô Trại lúc đang đi tham_khảo đất_nước rừng ở xã Lăng thì phát_hiện ra cây ba_kích ở trong một hốc núi Cả huyện trồng ba_kích Thấy mô_hình của Bhríu Pố hiệu_quả nhiều người dân kể_cả cán_bộ các huyện miền_núi khác như A_Lưới Thừa_Thiên_Huế Trà_My Nam_Giang Quảng_Nam đến học_hỏi Không chỉ trồng ươm giống tại hợp_tác_xã huyện Tây_Giang còn khuyến_khích người dân trồng loại cây này trên cơ_sở huyện sẽ cung_cấp giống miễn_phí gia_đình nào có _NUM ha ba_kích còn được thưởng _NUM triệu đồng Thậm chí họ còn bảo Bhríu Pố bị điên vì Đây là cây của trời mọc trong rừng chứ không phải của người không trồng được Nhưng khi bàn_bạc với mọi người trong thôn ai cũng phản_đối Bán tin bán nghi mình đào lên thì thấy có củ thật Ba_kích ưa bóng_râm nên trên đồi ba_kích Bhríu Pố trồng xen_kẽ cây sắn cau kiểng và cây ăn quả Qua tìm_hiểu sách_vở Bhríu Pố mới biết hết công_dụng tuyệt_vời của ba_kích"""
tn_127 = vi_clean(tn_127)
tn_127_vec = model.infer_vector(tn_127.split())
for tag,v in model.docvecs.most_similar([tn_127_vec]):
    print tag,v

r131_VN_92086 0.661446213722
r131_VN_148027 0.658583521843
r131_VN_168294 0.652355611324
r131_VN_147212 0.640873730183
r131_VN_165805 0.63979268074
r131_VN_158169 0.638164460659
r131_VN_175287 0.633736729622
r131_VN_127757 0.631079554558
r131_VN_142804 0.623704433441
r131_VN_132493 0.622422516346


In [14]:
zn_127 = u"""Ngoài ra Mùa_xuân đầu_tiên còn có câu_chuyện của thế_hệ trẻ Việt_Nam hôm_nay những người chịu trách_nhiệm bảo_vệ hòa_bình và dựng_xây đất_nước khi nhớ lại những chiến_công hào_hùng của dân_tộc Ngày _NUM miền_Nam Việt_Nam hoàn_toàn được giải_phóng Mùa_xuân đầu_tiên là chương_trình nghệ_thuật đặc_biệt là khúc tráng_ca gửi đến biết bao thế_hệ người Việt_Nam đã anh_hùng chiến_đấu vì độc_lập và tự_do cho dân_tộc Câu_chuyện và cảm_xúc của các nhân_chứng trong chiến_tranh như phóng_viên chiến_trường Patrick Chauvel ông Trần Quang Cơ nguyên thành_viên phái_đoàn đàm_phán Hiệp_định Hòa_bình Paris Đại_tướng Lê Đức Anh cũng được chia_sẻ Chiến_tranh đã lùi xa đất_nước hòa_bình thống_nhất song ký_ức về những ngày_tháng hành_quân ra_trận cùng chiến_thắng hào_hùng của quân và dân ta trong Đại_thắng mùa_xuân _NUM vẫn luôn khắc sâu trong tim mỗi người Việt_Nam Chương_trình được tổ_chức tại hai đầu cầu Nhà thi_đấu Nguyễn Du TP HCM và Trường_quay ngoài_trời của Đài truyền_hình Việt_Nam tại Hà_Nội với sự tham_gia của nhiều nghệ_sĩ ca_sĩ nổi_tiếng như Trọng Tấn Văn Mai Hương Nguyên Thảo Uyên Linh Mùa_xuân đầu_tiên sẽ được phát_sóng trực_tiếp trên VTV_NUM vào lúc _NUMh ngày _NUM Truyền_hình VTV tổ_chức cầu truyền_hình k niệm Giải_phóng miền_Nam Nhân dịp k niệm _NUM năm giải_phóng miền_Nam thống_nhất Đất_nước Đài truyền_hình Việt_Nam thực_hiện cầu truyền_hình trực_tiếp _NUM miền_Nam Bắc mang tên Mùa_xuân đầu_tiên Ảnh AP Trong chương_trình khán_giả sẽ được gặp nhiều khách_mời là các chiến_sĩ cách_mạng những người từng đi qua chiến_tranh với biết bao xương_máu và nước mắt và cả nhiều người con xa quê_hương cố_gắng trở_về trong ngày độc_lập Bình_thường Thông_tin hữu_ích nội_dung không lỗi Thông_tin nhanh nội_dung trung_thực và khách_quan Kém Nội_dung không có giá_trị nhiều lỗi Non_sông Việt_Nam quy về một mối sau _NUM năm chia_cắt Bài_viết được cập_nhật lần cuối lúc _NUM Tiểu Uyên VTV_NUM Giải_phóng miền_Nam thống_nhất đất_nước ĐÁNH_GIÁ BÀI_VIẾT Giúp Zing vn ngày_càng nâng_cao chất_lượng nội_dung bằng cách cho chúng_tôi biết bạn đánh_giá thế nào về bài_viết này Rất tốt"""
zn_127 = vi_clean(zn_127)
print zn_127
print 
zn_127_vec = model.infer_vector(zn_127.split())
for tag,v in model.docvecs.most_similar([zn_127_vec]):
    print tag,v

Ngoài ra Mùa_xuân đầu_tiên còn có câu_chuyện của thế_hệ trẻ Việt_Nam hôm_nay những người chịu trách_nhiệm bảo_vệ hòa_bình và dựng_xây đất_nước khi nhớ lại những chiến_công hào_hùng của dân_tộc Ngày miền_Nam Việt_Nam hoàn_toàn được giải_phóng Mùa_xuân đầu_tiên là chương_trình nghệ_thuật đặc_biệt là khúc tráng_ca gửi đến biết bao thế_hệ người Việt_Nam đã anh_hùng chiến_đấu vì độc_lập và tự_do cho dân_tộc Câu_chuyện và cảm_xúc của các nhân_chứng trong chiến_tranh như phóng_viên chiến_trường Patrick Chauvel ông Trần Quang Cơ nguyên thành_viên phái_đoàn đàm_phán Hiệp_định Hòa_bình Paris Đại_tướng Lê Đức Anh cũng được chia_sẻ Chiến_tranh đã lùi xa đất_nước hòa_bình thống_nhất song ký_ức về những ngày_tháng hành_quân ra_trận cùng chiến_thắng hào_hùng của quân và dân ta trong Đại_thắng mùa_xuân vẫn luôn khắc sâu trong tim mỗi người Việt_Nam Chương_trình được tổ_chức tại hai đầu cầu Nhà thi_đấu Nguyễn Du TP HCM và Trường_quay ngoài_trời của Đài truyền_hình Việt_Nam tại Hà_Nội với sự tham_gia củ

In [15]:
new_doc = u"chúng_tôi là nhà đầu_tư chân_chính"
new_v = model.infer_vector(new_doc.split())
for w,v in model.docvecs.most_similar([new_v]):
    print w,v

r131_VN_126327 0.51128077507
r131_VN_69112 0.493099182844
r131_VN_38161 0.491891473532
r131_VN_127156 0.489333033562
r131_VN_166597 0.48801240325
r131_VN_193618 0.483516275883
r131_ZN_5897 0.482938170433
r131_VN_142148 0.480082720518
r131_VN_95737 0.478205084801
r131_VN_87570 0.473291993141


## Model

In [17]:
from sklearn.externals import joblib

In [18]:
lr = joblib.load("/home/laampt/orm/10K/5K/model/lr_30Ktrain_cv10_r131.pkl")
rf = joblib.load("/home/laampt/orm/10K/5K/model/rf_30Ktrain_cv10_r131.pkl")
svc = joblib.load("/home/laampt/orm/10K/5K/model/svc_30Ktrain_cv10_r131.pkl")

In [25]:
print lr
print rf
print svc

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
            oob_score=False, random_state=127, verbose=0, warm_start=False)
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=127, tol=0.0001,
     verbose=0)


In [26]:
print type(lr).__name__

LogisticRegression


In [27]:
def voting_result3(test_doc, cls):
    test_vec = model.infer_vector(test_doc.split())
    results = [(type(clf).__name__, clf.predict(test_vec)) for clf in cls]
    return results
                
def print_sum_test_vec(test_doc):
    test_vec = model.infer_vector(test_doc.split())
    return np.sum(test_vec)

def voting_vec3(test_vec, cls):
    return [(type(clf).__name__, clf.predict(test_vec)) for clf in cls]

In [28]:
INPUT = u"""Nguỵ biện nói cho cùng là vấn đề chân lí và sự thật. Vấn đề ở VN là chân lí nó được bóp méo và vặn vẹo làm cho người dân nghĩ rằng chỉ có một chân lí duy nhất. Quan sát trên báo chí, internet, đến nghị trường, từ những người có học (cỡ sư sĩ), đến chính khách, và cả thường dân ai ai cũng dùng những lí lẽ mà họ không biết là lỗi nguỵ biện. Chẳng hạn như gần đây nhất có người cảnh báo rằng coi chừng lợi dụng dân chủ, nhưng trong thực tế đó cũng là một dạng của nguỵ biện – nguỵ biện dựa vào nguồn tin vu vơ. Còn sự thật thì cũng bị giả tạo và giả dối sản sinh (như vụ Lê Văn Tám). Những thói nguỵ biện nó được xây dựng trên sự dối trá đã được tạo ra quá lâu, nên khó có thể chỉnh sửa một sớm một chiều. Chỉ khi nào nền giáo dục có sự tham gia bình đẳng từ các nhóm xã hội, thay vì bị sự kiểm soát của một thế lực chính trị, thì tình trạng nguỵ biện vẫn còn."""
test_doc = vi_clean(get_tokens(INPUT))
print voting_result3(test_doc, [lr,rf,svc])

INFO:requests.packages.urllib3.connectionpool:Starting new HTTP connection (1): vlsp.vietlp.org
DEBUG:requests.packages.urllib3.connectionpool:"POST /demo/?page=seg_pos_chunk HTTP/1.1" 200 None


[('LogisticRegression', array([ 0.])), ('RandomForestClassifier', array([ 0.])), ('LinearSVC', array([ 0.]))]


In [57]:
INPUT = u"""Các nhà đối_kháng dân_chủ tiếp_tục bị hăm_dọa sách_nhiễu tống_ngục chỉ vì
 dám lên_tiếng phê_phán những sai_lầm và tố_cáo những tội_ác của nhà_cầm_quyền_Cộng_sản. Điển_hình là nhà_báo tự_do Nguyễn_Hoàng Hải tiếp_tục bị_giam sau
 khi mãn án_tù và nay biệt vô âm_tín giáo_sư hồi_hương"""
test_doc = vi_clean(vi_strip_text2(INPUT))
print voting_result3(test_doc, [lr,rf,svc])

[('LogisticRegression', array([ 0.])), ('RandomForestClassifier', array([ 0.])), ('LinearSVC', array([ 0.]))]


## Testing on NEWSWIRE

In [30]:
START_NUM = 30000
TEST_NUM = 3000
wrong_dim_count = 0

vne_posts = []
for i in range(START_NUM, START_NUM + TEST_NUM):
    if i % 3 == 0:
        k = "r131_VN_{}".format(i)
    elif i % 3 == 1:
        k = "r131_TN_{}".format(i)
    else:
        k = "r131_ZN_{}".format(i)
        
    v = model.docvecs[k]
    size = v.shape
    if size[0] == 200: 
        vne_posts.append((k,v))
    else:
        wrong_dim_count += 1

In [31]:
print wrong_dim_count
print len(vne_posts)

0
3000


In [51]:
results = []
for i, (k, vec) in enumerate(vne_posts[:3000]):
    if i % 100 == 0: print "[*] Vec ", k
    results.append(list(voting_vec3(vec, [lr,rf,svc])))

[*] Vec  r131_VN_30000
[*] Vec  r131_TN_30100
[*] Vec  r131_ZN_30200
[*] Vec  r131_VN_30300
[*] Vec  r131_TN_30400
[*] Vec  r131_ZN_30500
[*] Vec  r131_VN_30600
[*] Vec  r131_TN_30700
[*] Vec  r131_ZN_30800
[*] Vec  r131_VN_30900
[*] Vec  r131_TN_31000
[*] Vec  r131_ZN_31100
[*] Vec  r131_VN_31200
[*] Vec  r131_TN_31300
[*] Vec  r131_ZN_31400
[*] Vec  r131_VN_31500
[*] Vec  r131_TN_31600
[*] Vec  r131_ZN_31700
[*] Vec  r131_VN_31800
[*] Vec  r131_TN_31900
[*] Vec  r131_ZN_32000
[*] Vec  r131_VN_32100
[*] Vec  r131_TN_32200
[*] Vec  r131_ZN_32300
[*] Vec  r131_VN_32400
[*] Vec  r131_TN_32500
[*] Vec  r131_ZN_32600
[*] Vec  r131_VN_32700
[*] Vec  r131_TN_32800
[*] Vec  r131_ZN_32900


In [52]:
import pandas as pd

In [53]:
results2 = [[r[0][1][0], r[1][1][0], r[2][1][0]] for r in results]
df_result = pd.DataFrame.from_records(np.asarray(results2), columns=["LR", "RF", "SVC"])
df_result.describe()

,LR,RF,SVC
count,3000.000000,3000.000000,3000.000000
mean,0.093667,0.026667,0.069333
std,0.291413,0.161134,0.254062
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000


## Testing on BLACK 30K +

In [54]:
START_NUM = 30000
TEST_NUM = 3000
wrong_dim_count = 0

black_posts = []
for i in range(START_NUM, START_NUM + TEST_NUM):
    k = "r131_PD_{}".format(i)
        
    v = model.docvecs[k]
    size = v.shape
    if size[0] == 200: 
        black_posts.append((k,v))
    else:
        wrong_dim_count += 1
        
print wrong_dim_count
print len(black_posts)

0
3000


In [56]:
results = []
for i, (k, vec) in enumerate(black_posts):
    if i % 100 == 0: print "[*] Vec ", k
    results.append(list(voting_vec3(vec, [lr,rf,svc])))

results2 = [[r[0][1][0], r[1][1][0], r[2][1][0]] for r in results]
df_result = pd.DataFrame.from_records(np.asarray(results2), columns=["LR", "RF", "SVC"])
df_result.describe()

[*] Vec  r131_PD_30000
[*] Vec  r131_PD_30100
[*] Vec  r131_PD_30200
[*] Vec  r131_PD_30300
[*] Vec  r131_PD_30400
[*] Vec  r131_PD_30500
[*] Vec  r131_PD_30600
[*] Vec  r131_PD_30700
[*] Vec  r131_PD_30800
[*] Vec  r131_PD_30900
[*] Vec  r131_PD_31000
[*] Vec  r131_PD_31100
[*] Vec  r131_PD_31200
[*] Vec  r131_PD_31300
[*] Vec  r131_PD_31400
[*] Vec  r131_PD_31500
[*] Vec  r131_PD_31600
[*] Vec  r131_PD_31700
[*] Vec  r131_PD_31800
[*] Vec  r131_PD_31900
[*] Vec  r131_PD_32000
[*] Vec  r131_PD_32100
[*] Vec  r131_PD_32200
[*] Vec  r131_PD_32300
[*] Vec  r131_PD_32400
[*] Vec  r131_PD_32500
[*] Vec  r131_PD_32600
[*] Vec  r131_PD_32700
[*] Vec  r131_PD_32800
[*] Vec  r131_PD_32900


,LR,RF,SVC
count,3000.000000,3000.000000,3000.000000
mean,0.971667,0.955333,0.941000
std,0.165951,0.206605,0.235664
min,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000
75%,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000


In [60]:
from IPython.display import HTML, IFrame

In [67]:
%%HTML
<iframe src="http://stable.ascii-flow.appspot.com/#Draw8835268555573233485/752933462" width=700 height=350></iframe>

## Test on 800

In [68]:
import codecs

In [70]:
with codecs.open("/home/laampt/orm/10K/5K/test/bad.txt.nodup", encoding="utf-8", mode="r") as fin:
    with codecs.open("/home/laampt/orm/10K/5K/test/bad.txt.nodup2", encoding="utf-8", mode="w") as fout:
        for ii, line in enumerate(fin):
            if ii % 1000 == 0: logger.info("Progress at: {}".format(ii))
            fout.write(vi_clean(vi_strip_text2(line))+'\n')

INFO:root:Progress at: 0


In [72]:
print vi_strip_text2(u"nghĩa_trang Cồn_Dầu.......wwNUMBERww giáo_dân")

nghĩa_trang Cồn_DầuwwNUMBERww giáo_dân


In [74]:
print vi_strip_text2(u"PHÙNG_QUANG_THANH GẶP THƯỜNG VẠN_TOÀN NGÀY wwDATEww. Tuy nhiên bản_tin mập_mờ")

PHÙNG_QUANG_THANH GẶP THƯỜNG VẠN_TOÀN NGÀY wwDATEww Tuy nhiên bản_tin mập_mờ


In [75]:
print vi_strip_text2(u"PHÙNG_QUANG_THANH ĐÃ CHẾT...TIN TỪ HÀ_NỘI XÁC_NHẬN")

PHÙNG_QUANG_THANH ĐÃ CHẾTTIN TỪ HÀ_NỘI XÁC_NHẬN


In [76]:
print vi_strip_text2(u"hơi kiệt...Không_gian")

hơi kiệtKhông_gian
